# Library: sp
Data: Philadelphia database. Source: .csv, .shp

In [ ]:
# Import libraries
library(sp)
library(rgdal)
library(rgeos)

In [ ]:
# Open files

# Open Philadelphia homicides database
path_hom <- "./data/PhillyHomicides/"
philly_hom <- readOGR(dsn=path_hom, "PhillyHomicides")

# Open Philadelphia education database
path_edu <- "./data/PhiladelphiaEduAttain.csv"
philly_edu <- read.csv(path_edu)

# Open Philadelphia region database
path <- "./data/Philly/"
philly <- readOGR(dsn=path, "PhillyTotalPopHHinc")

In [ ]:
# Display columns names
names(philly_edu)

In [ ]:
# Display info about datasets
str(philly)
str(philly_hom)

In [ ]:
# Median household income
spplot(philly, "medHHinc", main="Mediana dochodu na mieszkanie")

In [ ]:
# Subset where household income is more than 40 000$
plot(philly, main="Obszary z dochodem powyżej 40 000$")
philly_40k <- subset(philly, medHHinc > 40000)
plot(philly_40k, add=T, col="blue", main="Obszary z dochodem powyżej 40 000$")

In [ ]:
# Total population
spplot(philly, "totalPop")

In [ ]:
# Display shape area of the first region
first_region_shape_area = philly[1, "Shape_area"]
spplot(first_region_shape_area, main="Pojedynczy region miasta")

In [ ]:
# Sampling
par(mfrow=c(2, 2))

set.seed(225)
regular <- spsample(first_region_shape_area, 100, type="regular", pretty=TRUE)
plot(regular, main="Próbkowanie regularne")

set.seed(225)
random <- spsample(first_region_shape_area, 100, type="random", pretty=TRUE)
plot(random, main="Próbkowanie losowe")

set.seed(225)
nonaligned <- spsample(first_region_shape_area, 100, type="nonaligned", main="Próbkowanie nieprzyległe")
plot(nonaligned, main="Próbkowanie nieprzyległe")

set.seed(225)
stratified <- spsample(first_region_shape_area, 100, type="stratified")
plot(stratified, main="Próbkowanie stratyfikowane")

In [ ]:
# Join database
philly_merge_edu <- merge(philly, philly_edu, by.x="GEOID10", by.y="GEOID")

# Dislay results
names(philly_merge_edu)

In [ ]:
# Reprojecting

# Check current coordinates
range(coordinates(philly))
range(coordinates(philly_hom))

proj4string(philly)
proj4string(philly_hom)

# Transport
philly_hom <- spTransform(philly_hom, CRS(proj4string(philly)))

# Check results
range(coordinates(philly))
range(coordinates(philly_hom))

proj4string(philly)
proj4string(philly_hom)

In [ ]:
# Spatial aggregation
philly_aggr <- aggregate(x=philly_hom["OBJ_ID"], by=philly, FUN=sum)

In [ ]:
# Number of murders in Phipadelphia
spplot(philly[,1], colorkey=TRUE, sp.layout=philly_hom["OBJ_ID"], main="Liczba morderstw w Filadelfii")
spplot(philly_aggr, main="Liczba morderstw w Filadelfii")
spplot(philly_hom["TEXT_GE"], main="Morderstwa w Filadelfii")